In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np
import cv2
import json
from tqdm import tqdm

from xml.dom import minidom

In [2]:
root = minidom.Document()

images = root.createElement('images')
dataset = root.createElement('dataset')
root.appendChild(dataset)

<DOM Element: dataset at 0x125e143a0>

In [3]:
df = pd.read_csv("data/ypr_df.csv", sep=";")

In [4]:
df["mouth_landmarks"] = df.mouth_landmarks.apply(lambda x: literal_eval(x))

In [5]:
for row in tqdm(df.itertuples()):
    img = cv2.imread(f"data/cropped_images/{row.name}")
    img_height, img_width = img.shape[:2]
    mouth_landmarks = np.array(row.mouth_landmarks).astype(np.uint32)
    left, top = mouth_landmarks.min(axis=0)-20
    right, bottom = mouth_landmarks.max(axis=0)+20
    mouth_width, mouth_height = right-left, bottom-top

    image = root.createElement('image')
    image.setAttribute('file', row.name)
    image.setAttribute('width', f'{img_width}')
    image.setAttribute('height', f'{img_height}')
    box = root.createElement('box')
    box.setAttribute('top', f'{top}')
    box.setAttribute('left', f'{left}')
    box.setAttribute('width', f'{mouth_width}')
    box.setAttribute('height', f'{mouth_height}')
    for landmark_idx, landmark_coords in enumerate(row.mouth_landmarks):
        part = root.createElement('part')
        part_name = f"{landmark_idx}" if landmark_idx>=10 else f"0{landmark_idx}"
        part.setAttribute('name', part_name)
        part.setAttribute('x', f'{mouth_landmarks[landmark_idx][0]}')
        part.setAttribute('y', f'{mouth_landmarks[landmark_idx][1]}')
        box.appendChild(part)
    image.appendChild(box)
    images.appendChild(image)

    # break

dataset.appendChild(images)

7674it [00:14, 533.45it/s]


<DOM Element: images at 0x125e14310>

In [6]:
xml_str = root.toprettyxml()

with open("training_with_mouth_landmarks.xml", "w") as f:
    f.write(xml_str)

In [47]:
top, bottom, left, right, mouth_width, mouth_height

(146, 211, 137, 242, 105, 65)

In [34]:
top, bottom = np.array(df.mouth_landmarks.values[0])[..., 1].min()-20, np.array(df.mouth_landmarks.values[0])[..., 1].max()+20
left, right = np.array(df.mouth_landmarks.values[0])[..., 0].min()-20, np.array(df.mouth_landmarks.values[0])[..., 0].max()+20

In [35]:
top, bottom, left, right

(146.4500911764706, 211.17877794117652, 137.85751617647054, 242.5200161764705)

In [38]:
mouth_landmarks = np.array(df.mouth_landmarks.values[0]).astype(np.uint32)
left, top = mouth_landmarks.min(axis=0)-20
right, bottom = mouth_landmarks.max(axis=0)+20
width, height = right-left, bottom-top

In [39]:
top, bottom, left, right, width, height

(146, 211, 137, 242, 105, 65)